# Release of hammer-cli gem

## Requirements
- push access to https://github.com/theforeman/hammer-cli
- push access to rubygems.org for hammer-cli
- sudo yum install transifex-client python-slugify asciidoc
- ensure neither the `git push` or `gem push` don't require interractive auth. If you can't use api key or ssh key to auth skip these steps and run them form the shell manually 
- to push translations you need an account on Transifex

## Release process
- Follow the steps with `<Shift>+<Enter>` or `<Ctrl>+<Enter>,<Down>`
- If anything fails, fix it and re-run the step if applicable

## Release settings

In [1]:
%cd ..

/home/mshira/git/hammer-cli


### Update the following notebook settings

In [2]:
NEW_VERSION = '2.0.0'
LAST_VERSION = '0.19.0'
DEVELOP_VERSION = '2.1.0-develop'
NEXT_FUTURE_VERSION = '2.1.0'
MAJOR_RELEASE = True
STABLE_BRANCH = '2.0-stable'
GIT_REMOTE_UPSTREAM = 'origin'
WORK_BRANCH = 'master' if MAJOR_RELEASE else STABLE_BRANCH

### Ensure the repo is up to date

In [3]:
! git checkout {WORK_BRANCH}

M	rel-eng/gem_release.ipynb
Already on 'master'


In [4]:
! git fetch {GIT_REMOTE_UPSTREAM}

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14 (delta 4), reused 8 (delta 4), pack-reused 0
Unpacking objects: 100% (14/14), done.
From github.com:theforeman/hammer-cli
   bc12a4d..d62870f  0.19-stable -> origin/0.19-stable
 * [new tag]         0.19.2      -> 0.19.2


In [6]:
! git rebase {GIT_REMOTE_UPSTREAM}/{WORK_BRANCH}

error: cannot rebase: You have unstaged changes.
error: Please commit or stash them.


### Run tests localy

In [5]:
! bundle update

Fetching gem metadata from https://rubygems.org/........
Resolving dependencies...
Using rake 10.1.1
Using json 2.3.0
Using oauth 0.5.4
Using http-accept 1.7.0
Using unf_ext 0.0.7.6
Using unf 0.1.4
Using domain_name 0.5.20190701
Using http-cookie 1.0.3
Using mime-types-data 3.2019.1009
Using mime-types 3.3.1
Using netrc 0.11.0
Using rest-client 2.1.0
Using apipie-bindings 0.3.0
Using awesome_print 1.8.0
Using debug_inspector 0.0.3
Using binding_of_caller 0.8.0
Using builder 3.2.4
Using bundler 1.17.3
Using byebug 11.1.1 (was 11.0.1)
Using ci_reporter 1.9.3
Using clamp 1.1.2
Using coderay 1.1.2
Using docile 1.3.2
Using fast_gettext 2.0.2 (was 2.0.1)
Fetching locale 2.1.3 (was 2.1.2)
Installing locale 2.1.3 (was 2.1.2)
Using text 1.3.1
Fetching gettext 3.3.4 (was 3.2.9)
Installing gettext 3.3.4 (was 3.2.9)
Using highline 2.0.3
Using little-plugger 1.1.4
Using multi_json 1.14.1
Using logging 2.2.2
Using unicode 0.4.4.4
Using unicode-display_width 1.6.1 (was 1.6.0)
Using hammer_cli 2.0.0.p

In [7]:
! bundle exec rake test

/home/mshira/.rvm/rubies/ruby-2.5.1/bin/ruby -I"lib:lib" -I"/home/mshira/.rvm/gems/ruby-2.5.1/gems/rake-10.1.1/lib" "/home/mshira/.rvm/gems/ruby-2.5.1/gems/rake-10.1.1/lib/rake/rake_test_loader.rb" "test/unit/messages_test.rb" "test/unit/exception_handler_test.rb" "test/unit/modules_test.rb" "test/unit/connection_test.rb" "test/unit/completer_test.rb" "test/unit/csv_parser_test.rb" "test/unit/option_builder_test.rb" "test/unit/utils_test.rb" "test/unit/apipie/command_test.rb" "test/unit/apipie/option_builder_test.rb" "test/unit/apipie/api_connection_test.rb" "test/unit/apipie/option_definition_test.rb" "test/unit/command_extensions_test.rb" "test/unit/main_test.rb" "test/unit/options/processor_list_test.rb" "test/unit/options/validators/dsl_test.rb" "test/unit/options/sources/command_line_test.rb" "test/unit/options/sources/saved_defaults_test.rb" "test/unit/options/matcher_test.rb" "test/unit/options/option_collector_test.rb" "test/unit/options/normalizers_test.rb" "test/unit/options/

### Update release related stuff

In [8]:
! sed -i 's/Gem::Version.new .*/Gem::Version.new "{NEW_VERSION}"/' lib/hammer_cli/version.rb

In [ ]:
# Parse git changelog
from IPython.display import Markdown as md
from subprocess import check_output
from shlex import split
import re

def format_log_entry(entry):
    issues = re.findall(r'[^(]#([0-9]+)', entry)
    entry = re.sub(r'([fF]ixes|[rR]efs)[^-]*-\s*(.*)', r'\2', entry)
    entry = '* ' + entry.capitalize()
    entry = re.sub(r'\(#([0-9]+)\)', r'([PR #\1](https://github.com/theforeman/hammer-cli/pull/\1))', entry)
    for i in issues:
        referenced_issues.append(i)
        entry = entry + ', [#%s](http://projects.theforeman.org/issues/%s)' % (i, i)
    return entry

def skip(entry):
    if re.match(r'Merge pull', entry) or \
      re.match(r'^i18n', entry) or \
      re.match(r'^Bump to version', entry):
        return True
    else:
        return False
referenced_issues = []    
git_log_cmd = 'git log --pretty=format:"%%s" %s..HEAD' % LAST_VERSION
log = check_output(split(git_log_cmd)).decode('utf8').split('\n')
change_log = [format_log_entry(e) for e in log if not skip(e)]
md('\n'.join(change_log))


In [ ]:
# Write release notes
from datetime import datetime
import fileinput
import sys

fh = fileinput.input('doc/release_notes.md', inplace=True)  
for line in fh:  
    print(line.rstrip())
    if re.match(r'========', line):
        print('### %s (%s)' % (NEW_VERSION, datetime.today().strftime('%Y-%m-%d')))
        for entry in change_log:
            print(entry)
        print('')
fh.close()  

#### Manual step: Update deps in the gemspec if neccessary

### Check what is going to be commited

In [ ]:
! git add -u
! git status

In [ ]:
! git diff --cached

### Commit changes

In [ ]:
! git commit -m "Bump to {NEW_VERSION}"

### Update translations

In [ ]:
if MAJOR_RELEASE:
    ! make -C locale/ tx-update

### Tag new version

In [ ]:
! git tag {NEW_VERSION}

### Prepare stable branch for major release

In [ ]:
if MAJOR_RELEASE:
    ! git checkout -b {STABLE_BRANCH}
    ! git push {GIT_REMOTE_UPSTREAM} {STABLE_BRANCH}
    ! git checkout {WORK_BRANCH}

### Build the gem

In [ ]:
! rake build

In [ ]:
! gem push pkg/hammer_cli-{NEW_VERSION}.gem

### Bump the develop version for major release

In [ ]:
if MAJOR_RELEASE:
    ! sed -i 's/Gem::Version.new .*/Gem::Version.new "{DEVELOP_VERSION}"/' lib/hammer_cli/version.rb

In [ ]:
if MAJOR_RELEASE:
    ! git add -u
    ! git status

In [ ]:
if MAJOR_RELEASE:
    ! git diff --cached

In [ ]:
if MAJOR_RELEASE:
    ! git commit -m "Bump to {DEVELOP_VERSION}"

### PUSH the changes upstream If everything is correct

In [ ]:
! git push {GIT_REMOTE_UPSTREAM} {WORK_BRANCH}

In [ ]:
! git push --tags {GIT_REMOTE_UPSTREAM} {WORK_BRANCH}

#### Now the new release is in upstream repo

### Some manual steps follow to improve the UX

#### New relase on GitHub

Copy the following changelog lines to the description in form on link below
The release title is the new version.

In [ ]:
print('\n')
print('\n'.join(change_log))
print('\n\nhttps://github.com/theforeman/hammer-cli/releases/new?tag=%s' % NEW_VERSION)

In [ ]:
from IPython.display import Markdown as md

md('### Create new hammer-cli release in Redmine \n' + \
    '<a href="https://projects.theforeman.org/projects/hammer-cli/versions/new" target="_blank">https://projects.theforeman.org/projects/hammer-cli/versions/new</a>\n\n' + \
    'Set name to hammer-cli-%s' % (NEXT_FUTURE_VERSION if MAJOR_RELEASE else NEW_VERSION))

In [ ]:
if not MAJOR_RELEASE:
    print('Set fixed in versions to %s in following issues:' % NEW_VERSION)
    for i in referenced_issues:
        print('- https://projects.theforeman.org/issues/%s' % i)

## Congratulations

Release is public now.